In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams.update({'font.size': 42})
import os
import textwrap
import csv
import json
import re
import json
import seaborn as sns
import pandas as pd
import tensorflow as tf

In [3]:
class HorizontalNetworkV5():
    def __init__(self, input_shape, optimizer, left_classif_factor, right_classif_factor, siamese_factor):
        self.input_shape = input_shape
        self.optimizer = optimizer
        self.left_classif_factor = left_classif_factor
        self.right_classif_factor = right_classif_factor
        self.siamese_factor = siamese_factor
        self.left_input = Input(self.input_shape, name="Left_input")
        self.right_input = Input(self.input_shape, name="Right_input")

    def build_net(self, num_outputs, quantization):
        def horizontal_block(input, block_name):
            conv0 = Conv2D(8, (9, 9), padding="same", activation='relu',
                           kernel_initializer="he_normal", name='{}_conv0'.format(block_name),
                           kernel_regularizer=l2(1e-2))(input)
            conv0 = BatchNormalization(name='{}_conv0_batchnorm'.format(block_name))(conv0)

            conv1 = Conv2D(8, (7, 7), padding="same", activation='relu',
                           kernel_initializer="he_normal", name='{}_conv1'.format(block_name),
                           kernel_regularizer=l2(1e-2))(input)
            conv1 = BatchNormalization(name='{}_conv1_batchnorm'.format(block_name))(conv1)

            conv2 = Conv2D(8, (5, 5), padding="same", activation='relu',
                           kernel_initializer="he_normal", name='{}_conv2'.format(block_name),
                           kernel_regularizer=l2(1e-2))(input)
            conv2 = BatchNormalization(name='{}_conv2_batchnorm'.format(block_name))(conv2)

            conv3 = Conv2D(8, (3, 3), padding="same", activation='relu',
                           kernel_initializer="he_normal", name='{}_conv3'.format(block_name),
                           kernel_regularizer=l2(1e-2))(input)
            conv3 = BatchNormalization(name='{}_conv3_batchnorm'.format(block_name))(conv3)

            return concatenate([conv0, conv1, conv2, conv3])

        net_input = Input(shape=self.input_shape)

        convnet = horizontal_block(net_input, "block0")

        convnet = horizontal_block(convnet, "block1")

        convnet = horizontal_block(convnet, "block2")

        convnet = horizontal_block(convnet, "block3")

        convnet = horizontal_block(convnet, "block4")
        convnet = MaxPooling2D(padding="valid")(convnet)

        convnet = horizontal_block(convnet, "block5")
        convnet = MaxPooling2D(padding="valid")(convnet)

        convnet = horizontal_block(convnet, "block6")
        convnet = MaxPooling2D(padding="valid")(convnet)

        convnet = horizontal_block(convnet, "block7")
        convnet = MaxPooling2D(padding="valid")(convnet)

        embedding_model = Model(inputs=net_input, outputs=convnet, name="horizontal_blocks")
        if quantization == "nullhop":
            embedding_model = apply_quantization(embedding_model)
        elif quantization == "edgetpu":
            embedding_model = tfmot.quantization.keras.quantize_model(embedding_model)
        elif quantization is None:
            pass
        else:
            raise ValueError

        encoded_l = embedding_model(self.left_input)
        encoded_r = embedding_model(self.right_input)

        common_concat = concatenate([encoded_l, encoded_r])
        common_input = Input(shape=common_concat.shape[1:])
        common_branch = Conv2D(512, (3, 3), padding="same", activation='relu',
                               kernel_initializer="he_normal", name='center_conv1',
                               kernel_regularizer=l2(1e-2))(common_input)
        common_branch = Flatten()(common_branch)
        common_branch = Dense(512, activation="relu", kernel_regularizer=l2(1e-3),
                              kernel_initializer="he_normal")(common_branch)
        common_branch = Dropout(0.5)(common_branch)
        siamese_prediction = Dense(1, activation='sigmoid',
                                   name="Siamese_classification_dense")(common_branch)

        siamese_model = Model(inputs=common_input, outputs=siamese_prediction, name="Siamese_classification")

        print(siamese_model)
        if quantization == "nullhop":
            siamese_model = apply_quantization(siamese_model)(common_concat)
        elif quantization == "edgetpu":
            siamese_model = tfmot.quantization.keras.quantize_model(siamese_model)(common_concat)
        elif quantization is None:
            siamese_model = siamese_model(common_concat)
        else:
            raise ValueError

        right_branch = Flatten()(encoded_r)
        right_branch = Dense(512, activation="relu", kernel_regularizer=l2(1e-2), kernel_initializer="he_normal",
                             name='right_dense0')(right_branch)
        right_branch_classif = Dense(num_outputs, activation='softmax', name="Right_branch_classification")(right_branch)

        left_branch = Flatten()(encoded_l)
        left_branch = Dense(512, activation="relu", kernel_regularizer=l2(1e-2), kernel_initializer="he_normal",
                            name='left_dense0')(left_branch)
        left_branch_classif = Dense(num_outputs, activation='softmax', name="Left_branch_classification")(left_branch)

        siamese_net = Model(inputs=[self.left_input, self.right_input],
                            outputs=[left_branch_classif, siamese_model, right_branch_classif])

        siamese_net.compile(loss={"Left_branch_classification": "categorical_crossentropy",
                                  "Siamese_classification": "binary_crossentropy",
                                  "Right_branch_classification": "categorical_crossentropy"},
                            optimizer=self.optimizer,
                            metrics={"Left_branch_classification": "accuracy",
                                     "Siamese_classification": "accuracy",
                                     "Right_branch_classification": "accuracy"},
                            loss_weights={"Left_branch_classification": self.left_classif_factor,
                                          "Siamese_classification": self.siamese_factor,
                                          "Right_branch_classification": self.right_classif_factor})
        return siamese_net

In [2]:
new_model = tf.keras.models.load_model('/mnt/Storage/code/low-shot/siamese_on_edge_tf2/results/2020_6_11-15_49_18_159616_seed_13_tiny-imagenet_HorizontalNetworkV5_quantization_None_classif_yes/saved_model')

NotImplementedError: When subclassing the `Model` class, you should implement a `call` method.